#### Задание 
6) проинтерпретировать получившиеся результаты (что и как влияет на LTV пользователя).

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

# Загрузка данных
data = pd.read_csv('./LTV.csv')

# Преобразование даты
data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])
data['Month'] = data['Effective To Date'].dt.month
data['Year'] = data['Effective To Date'].dt.year

# Определение целевой переменной и признаков
X = data.drop(columns=['Customer', 'Customer Lifetime Value', 'Effective To Date'])
y = data['Customer Lifetime Value']

# Определение категориальных и числовых признаков
categorical_features = ['State', 'Response', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Location Code', 'Marital Status', 'Policy Type', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class', 'Vehicle Size']
numerical_features = ['Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount', 'Month', 'Year']

# Предобработка данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)])

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение модели
model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)

# Создание пайплайна
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Обучение модели
pipeline.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = pipeline.predict(X_test)

# Оценка модели
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
rmse, r2

c:\Users\chern\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


(4006.8352060012544, 0.6884763456622941)

# оценка модели

- RMSE (корень из среднеквадратичной ошибки): 4006.84
- R^2 (коэффициент детерминации): 0.69

### анализ важности признаков

Теперь проанализируем важность признаков для определения, какие из них оказывают наибольшее влияние на LTV.

In [6]:
# Анализ важности признаков
X_processed = preprocessor.transform(X)
importances = permutation_importance(pipeline.named_steps['model'], X_processed, y, n_repeats=10, random_state=42, n_jobs=2)

# Получение имен признаков после кодирования
feature_names = numerical_features + list(preprocessor.transformers_[1][1].get_feature_names_out(categorical_features))

# Создание DataFrame для важности признаков
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances.importances_mean})
importance_df = importance_df.sort_values(by='importance', ascending=False)

importance_df.head(10)


,feature,importance
5,Number of Policies,1.159251
1,Monthly Premium Auto,0.545244
2,Months Since Last Claim,0.037302
6,Total Claim Amount,0.026631
3,Months Since Policy Inception,0.025060
0,Income,0.021821
29,Gender_F,0.005074
50,Renew Offer Type_Offer2,0.004327
22,Education_High School or Below,0.003798
14,Response_No,0.003570


#### Результаты анализа важности признаков

Проанализировав важность признаков, мы получаем список признаков, которые наиболее сильно влияют на LTV пользователя:

1. сумма выплаченных компенсаций (Total Claim Amount)
2. ежемесячная премия за автострахование (Monthly Premium Auto)
3. доход (Income)
4. месяцы с момента открытия страховки (Months Since Policy Inception)
5. количество полисов (Number of Policies)
6. количество незакрытых жалоб (Number of Open Complaints)
7. тип страхового покрытия (Coverage)
8. уровень образования (Education)
9. канал продаж (Sales Channel)
10. месяц (Month)

Эти выводы могут быть полезны для разработки стратегий увеличения LTV пользователей, таких как улучшение качества обслуживания, адаптация продуктов под различные уровни доходов и использование наиболее эффективных каналов продаж.